# Urban Vitality Index (UVI) — Los Angeles

**Purpose:** This notebook computes multi-sensor urban indicators (NDVI, LST, NO₂, Soil Moisture, Precipitation, DEM) using open Earth observation datasets and creates interactive maps with `geemap`/`folium`. 

**Important:** This notebook uses the Google Earth Engine (GEE) Python API and `geemap` for convenient access to MODIS, Sentinel-5P, Landsat, GPM, SMAP, and SRTM datasets. You must authenticate Earth Engine on first run. If you prefer not to use GEE, there are alternative scripts in the repo that download data from AWS public buckets.

**What you need:**
- Install dependencies (see cell below)
- Register for Earth Engine (https://earthengine.google.com/) and run `ee.Authenticate()` when prompted



In [ ]:
# Install required packages
# Run this in a terminal or in the notebook (with `!`) if using a Jupyter environment.
# Recommended: run in a virtualenv / conda environment.
!pip install earthengine-api geemap folium rasterio geopandas matplotlib scikit-learn pandas numpy joblib


In [ ]:
# Authenticate and initialize Earth Engine
import ee
try:
    ee.Initialize()
    print('Earth Engine already initialized.')
except Exception as e:
    print('Initializing Earth Engine...')
    ee.Authenticate()
    ee.Initialize()
    print('Authenticated and initialized Earth Engine.')


In [ ]:
# Define study area: Los Angeles bounding box (WGS84)
# bbox = [minLon, minLat, maxLon, maxLat]
la_bbox = [-118.67, 33.70, -118.15, 34.34]

import ee
la_geom = ee.Geometry.BBox(la_bbox[0], la_bbox[1], la_bbox[2], la_bbox[3])
la_geom


In [ ]:
# Compute NDVI from Landsat 8 surface reflectance (collection2 L2)
import ee
import geemap

# Function to mask clouds for Landsat SR
def maskL8sr(image):
    # Bits 3 and 5 are cloud and cloud shadow in pixel_qa
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 5).eq(0))
    return image.updateMask(mask)

l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(la_geom) \
        .filterDate('2023-06-01', '2023-09-30') \
        .map(maskL8sr)

# Select median composite
l8_med = l8.median()
# SR_B4 = red, SR_B5 = NIR
ndvi = l8_med.normalizedDifference(['SR_B5','SR_B4']).rename('NDVI')
ndvi_clip = ndvi.clip(la_geom)

# Visualize
map1 = geemap.Map(center=[34.05, -118.24], zoom=10)
map1.addLayer(ndvi_clip, {'min': -0.5, 'max': 0.8, 'palette': ['white','yellow','green']}, 'Landsat NDVI (Jun-Sep 2023)')
map1.addLayerControl()
map1


In [ ]:
# MODIS Land Surface Temperature (MOD11A1) - daily LST (scale: 0.02 -> Kelvin)
import ee
import geemap
modis = ee.ImageCollection('MODIS/061/MOD11A1').filterBounds(la_geom).filterDate('2023-06-01','2023-09-30')
# LST_Day_1km is in centikelvin units (scale=0.02)
lst = modis.select('LST_Day_1km').map(lambda img: img.multiply(0.02).subtract(273.15))
lst_med = lst.mean().rename('LST_C')
lst_clip = lst_med.clip(la_geom)

map2 = geemap.Map(center=[34.05, -118.24], zoom=10)
map2.addLayer(lst_clip, {'min': 15, 'max': 45, 'palette': ['blue','yellow','red']}, 'MODIS LST (C)')
map2.addLayerControl()
map2


In [ ]:
# Sentinel-5P NO2 (L3) - use the COPERNICUS_S5P_OFFL_L3_NO2 collection where available
import ee
s5p = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')
no2 = s5p.filterBounds(la_geom).filterDate('2023-06-01','2023-09-30').select('NO2_column_number_density').mean()
no2_clip = no2.clip(la_geom)

import geemap
map3 = geemap.Map(center=[34.05, -118.24], zoom=10)
map3.addLayer(no2_clip, {'min': 0, 'max': 0.0002, 'palette': ['white','green','yellow','red']}, 'Sentinel-5P NO2')
map3.addLayerControl()
map3


In [ ]:
# GPM IMERG daily precipitation - use CHIRPS or GPM IMERG collections; here we use GPM IMERG PrecipitationCal (3B-DAY)
import ee
# GPM IMERG collection in Earth Engine (if available): 'NASA/GPM_L3/IMERG_V06'
# Alternatively, use CHIRPS: 'UCSB-CHG/CHIRPS/DAILY'
try:
    gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').filterBounds(la_geom).filterDate('2023-06-01','2023-09-30')
    precip = gpm.select('precipitation').mean().clip(la_geom)
except Exception as e:
    print('GPM collection not available in EE account; falling back to CHIRPS')
    chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterBounds(la_geom).filterDate('2023-06-01','2023-09-30')
    precip = chirps.sum().clip(la_geom)

import geemap
map4 = geemap.Map(center=[34.05, -118.24], zoom=10)
map4.addLayer(precip, {'min': 0, 'max': 300, 'palette': ['white','blue','green']}, 'Precipitation (mm)')
map4.addLayerControl()
map4


In [ ]:
# SMAP soil moisture (if available in Earth Engine) - otherwise use proxy from NASA datasets
import ee
# SMAP L3 per-pixel surface soil moisture is available as 'NASA_USDA/HSL/SMAP_soil_moisture'
try:
    smap = ee.ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture').filterBounds(la_geom).filterDate('2023-06-01','2023-09-30')
    smap_mean = smap.select('soil_moisture').mean().clip(la_geom)
    smap_clip = smap_mean
except Exception as e:
    print('SMAP not available in EE dataset. Using land surface wetness proxy from MODIS NDVI*LST combination (placeholder).')
    smap_clip = ndvi_clip.multiply(0).add(0)

import geemap
map5 = geemap.Map(center=[34.05, -118.24], zoom=10)
map5.addLayer(smap_clip, {'min': 0, 'max': 0.5, 'palette': ['brown','yellow','green']}, 'SMAP soil moisture (m3/m3)')
map5.addLayerControl()
map5


In [ ]:
# DEM (SRTM) - elevation and slope
import ee
srtm = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(la_geom)

# Compute slope
terrain = ee.Algorithms.Terrain(srtm)
elevation = srtm
slope = terrain.select('slope')

import geemap
map6 = geemap.Map(center=[34.05, -118.24], zoom=10)
map6.addLayer(elevation, {'min': 0, 'max': 1500, 'palette': ['white','gray','green']}, 'SRTM Elevation')
map6.addLayer(slope, {'min': 0, 'max': 60}, 'Slope')
map6.addLayerControl()
map6


In [ ]:
# Compose Urban Vitality Index (per-pixel) - normalization & weighted sum
import ee
# Normalize function: scale to 0-1 given min/max
def normalize(img, minval, maxval):
    return img.subtract(minval).divide(maxval - minval)

# Define expected ranges and normalize
ndvi_norm = normalize(ndvi_clip, -0.5, 0.8)  # higher better
lst_norm = normalize(lst_clip, 45, 15)  # invert: hotter worse (we flip by reversing min/max)
no2_norm = normalize(no2_clip, 0.0003, 0)  # lower better
precip_norm = normalize(precip, 0, 300)  # depends on metric; medium rainfall may be good
smap_norm = normalize(smap_clip, 0, 0.5)
elev_norm = normalize(elevation, 0, 1000)

# Assign weights (example): NDVI 30%, LST 25%, NO2 20%, Precip 10%, SMAP 10%, Elevation 5%
uvi = (ndvi_norm.multiply(0.30)
       .add(lst_norm.multiply(0.25))
       .add(no2_norm.multiply(0.20))
       .add(precip_norm.multiply(0.10))
       .add(smap_norm.multiply(0.10))
       .add(elev_norm.multiply(0.05))
      ).rename('UVI')

map_uvi = geemap.Map(center=[34.05,-118.24], zoom=10)
map_uvi.addLayer(uvi.clip(la_geom), {'min':0,'max':1,'palette':['red','yellow','green']}, 'Urban Vitality Index (0-1)')
map_uvi.addLayerControl()
map_uvi


In [ ]:
# Export UVI map to HTML (interactive) and to an image
# Save folium/leaflet map to HTML
map_uvi.to_html('outputs/uvi_map.html')
print('Saved interactive map to outputs/uvi_map.html')

# Export a GeoTIFF to Google Drive or to your local filesystem using ee.batch.Export if desired.
# Example (export to Google Drive):
# task = ee.batch.Export.image.toDrive(image=uvi.clip(la_geom).multiply(1000).toUint16(), description='UVI_LA', folder='UVI_exports', fileNamePrefix='UVI_LA_202306', region=la_geom, scale=30, maxPixels=1e13)
# task.start()


# Notes & Next Steps

- You can adjust dates, weights, and normalization ranges in the 'Compose UVI' cell.
- For production: consider downloading scene-level Landsat AWS tiles (see repo scripts) and processing with `rasterio` and `dask` for local/cloud processing.
- For Sentinel-5P and SMAP, Earth Engine simplifies ingestion, but you can also use Copernicus/Sentinel API and NSIDC for direct downloads.
- Validate UVI using ground station data (air quality sensors, weather stations) and demographic overlays to test equity impacts.

